# Simulated Annealing for the Job Shop Scheduling Problem

Simulated annealing is a metaheuristic optimization technique inspired by the process of annealing in metallurgy. In the context of the JSSP, it's used to find a schedule that minimizes a certain objective, typically the makespan (the total time required to complete all jobs).

The process can be summarized as follows:

1.  **Start with an initial solution:** We begin with a valid, but likely suboptimal, schedule. This initial solution can be generated using a simple heuristic, like a dispatching rule.
2.  **Define an "energy" function:** In our case, the energy of a schedule is its makespan, plus penalties for any violated constraints (like deadlines). A lower energy means a better solution.
3.  **Iteratively explore neighbors:** The algorithm then iteratively explores "neighboring" solutions. A neighbor is a new schedule created by making a small change to the current one, for example, by swapping the order of two operations on a single machine.
4.  **Accept or reject new solutions:**
      * If the neighbor solution has a lower energy (is better), it is always accepted as the new current solution.
      * If the neighbor solution has a higher energy (is worse), it might still be accepted with a certain probability. This probability is higher at the beginning (when the "temperature" is high) and decreases over time. This allows the algorithm to escape local optima and explore a wider range of solutions.
5.  **Cooling down:** The "temperature" gradually decreases, reducing the probability of accepting worse solutions. The process stops when the system has "cooled down" (the temperature is low) or after a certain number of iterations.

## Core Components

The implementation is structured into a few key components:

  * **`SimulatedAnnealingSolver`:** This is the main high-level solver class that you will interact with. It wraps the core annealing logic and provides a simple `solve` method.
  * **`JobShopAnnealer`:** This is a helper class that inherits from the `simanneal` library's `Annealer` class. It implements the core simulated annealing logic, including the `move` and `energy` functions.
  * **Neighbor Generators:** These are functions that define how to create a "neighbor" schedule from a current one.
  * **Objective Functions:** The `energy` function calculates the objective value of a schedule, which is typically the makespan plus any penalties for constraint violations.

## The `SimulatedAnnealingSolver`

This is the main entry point for using the solver. When you create an instance of this class, you can configure various parameters of the annealing process:

  * `initial_temperature` (`Tmax`): The starting temperature. A higher value increases the initial probability of accepting worse solutions.
  * `ending_temperature` (`Tmin`): The final temperature. The algorithm stops when it approaches this temperature.
  * `steps`: The total number of iterations to run.
  * `neighbor_generator`: The function used to generate neighboring solutions. The default is `swap_in_critical_path`.
  * `seed`: A random seed for reproducibility.

## Neighbor Generation Strategies

A key part of the simulated annealing process is how you explore the solution space by moving from one solution to a "neighbor". This implementation provides three different neighbor generation strategies in `_neighbor_generators.py`:

1.  **`swap_adjacent_operations`:** This function randomly selects a machine and swaps two adjacent operations in its sequence. This is a very localized search.

2.  **`swap_random_operations`:** This function randomly selects a machine and swaps two *random* operations in its sequence. This allows for larger jumps in the solution space compared to adjacent swaps.

3.  **`swap_in_critical_path` (Default):** This is the most sophisticated of the three. It identifies the critical path of the current schedule (the sequence of operations that determines the makespan) and looks for consecutive operations on that path that are on the same machine. It then swaps one of these pairs. The idea is that modifying the critical path is the most direct way to try to reduce the makespan. If no such pair exists, it falls back to a standard adjacent swap.

## The Objective Function

The objective function is what the simulated annealing algorithm tries to minimize. In the context of job shop scheduling, this is typically the makespan (the total time to complete all jobs) plus any penalties for violating constraints (like deadlines).

## Examples

Here are some examples of how to use the solver.

### Basic Usage

This example shows how to solve a benchmark instance ("ft06") with a specific seed to get a reproducible result.

In [1]:
from job_shop_lib.benchmarking import load_benchmark_instance
from job_shop_lib.metaheuristics import SimulatedAnnealingSolver


ft06_instance = load_benchmark_instance("ft06")

solver = SimulatedAnnealingSolver(
    seed=42,  # Seed for reproducibility
    initial_temperature=10,  # Low temperature --> more greedy search
    steps=1000,  # The number of schedules to try before stopping
    updates=100,  # How often the display table is updated
)

schedule = solver.solve(ft06_instance)
makespan = schedule.makespan()
print(f"The makespan found for ft06 is: {makespan}")

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     2.50000         60.00    70.00%    20.00%     0:00:00     0:00:00

The makespan found for ft06 is: 55


### Using a Different Neighbor Generator

Although it's not recommended, you can easily plug in a different neighbor generation strategy by passing it to the `SimulatedAnnealingSolver`'s constructor. Here's how to use `swap_adjacent_operations`:


In [2]:
from job_shop_lib.metaheuristics import swap_adjacent_operations

# This time, we specify a different neighbor generator
solver_adj_swap = SimulatedAnnealingSolver(
    seed=42,
    initial_temperature=10,
    steps=1000,
    updates=100,
    neighbor_generator=swap_adjacent_operations,
)

schedule_adj_swap = solver_adj_swap.solve(ft06_instance)
makespan_adj_swap = schedule_adj_swap.makespan()
print(f"The makespan for ft06 with adjacent swaps is: {makespan_adj_swap}")

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     2.50000         67.00   100.00%    20.00%     0:00:03     0:00:00

The makespan for ft06 with adjacent swaps is: 61


Similarly, you can use `swap_random_operations`:

In [3]:
from job_shop_lib.metaheuristics import swap_random_operations

solver_rand_swap = SimulatedAnnealingSolver(
    seed=42,
    initial_temperature=10,
    steps=1000,
    updates=100,
    neighbor_generator=swap_random_operations,
)
schedule_rand_swap = solver_rand_swap.solve(ft06_instance)
makespan_rand_swap = schedule_rand_swap.makespan()
print(f"The makespan for ft06 with random swaps is: {makespan_rand_swap}")

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     2.50000         70.00    50.00%     0.00%     0:00:09     0:00:00

The makespan for ft06 with random swaps is: 61


In both cases, the algorithm doesn't find a better solution than the starting one (found with the Most Work Remaining dispatching rule). Additionally, while using these alternative neighbor generators, you may notice that each step takes longer to compute. This happens because swapping operations randomly often results in unfeasible schedules. If the schedule is invalid, the neighbor generator will try again until it finds a valid neighbor.

We allow the possibility of customizing the neighbor generation strategy because for schedules with deadlines or due dates, the default `swap_in_critical_path` may not always yield the best results. In such cases, creating a custom neighbor generator that swaps operations that violate deadlines can be beneficial.

### Handling Deadlines

The solver can handle constraints like deadlines. The `energy` function will add a large penalty for any schedule where an operation finishes after its deadline. This guides the search towards solutions that respect the deadlines. You can specify how to penalize these violations with the `due_date_penalty_factor` and the `deadline_penalty_factor` parameters when creating the `SimulatedAnnealingSolver`.

In [4]:
from job_shop_lib.metaheuristics import get_makespan_with_penalties_objective


solver = SimulatedAnnealingSolver(
    seed=42,
    initial_temperature=10,
    steps=1000,
    updates=100,
    objective_function=get_makespan_with_penalties_objective(
        deadline_penalty_factor=10_000,  # We can tune thes# Solve the probleme factors here
        due_date_penalty_factor=100,
    ),  # Since there are no extra constraints in ft06, this returns the makespan
)

# Solve the problem
schedule = solver.solve(ft06_instance)

makespan = schedule.makespan()
print(f"The makespan found for ft06 is: {makespan}")

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     2.50000         60.00    70.00%    20.00%     0:00:00     0:00:00

The makespan found for ft06 is: 55
